In [9]:
import sys, os

# # Add the src directory to sys.path
# src_path = os.path.join(os.path.dirname(__file__), 'src')
# sys.path.append(src_path)

import numpy as np
import pandas as pd
import shelve
from src.neuron import *
from src.utils import *
from src.constants import * 
from src.network import *
from src.validation import *
from src.viz import *
from src.genetic_algorithm import *
from copy import copy
from datetime import datetime


In [ ]:
import pickle
import pandas as pd

file_path1 = './data/run_data.pkl'
file_path2 = '/Users/stevenwendel/Documents/GitHub/bg/data/2024-12-01_23-59-24.pkl'

def pkl_to_dataframe(file_path: str) -> pd.DataFrame:
    data = np.array()
    with open(file_path, 'rb') as file:
        try:
            while True:
                item = pickle.load(file)
                data= np.append(data, item)
        except EOFError:
            pass
    df = pd.DataFrame(data)
    return df

# Load and analyze file_path2
df = pkl_to_dataframe(file_path2)
df

In [7]:
import pickle
import pandas as pd

file_path2 = '/Users/stevenwendel/Documents/GitHub/bg/data/2024-12-01_23-59-24.pkl'

data = pickle.load(open(file_path2, 'rb'))
ga_results = pd.DataFrame()
for gen in data.keys():
    for iteration in data[gen].keys():
        # print(f'Currently looking at: {gen}-{iteration}')
        restructured_dict = {}
        restructured_dict['generation'] = gen
        restructured_dict['iteration'] = iteration
        restructured_dict['dna'] = data[gen][iteration]['dna']
        restructured_dict['dna_score'] = data[gen][iteration]['dna_score']
        restructured_dict['neuron_data'] = data[gen][iteration]['neuron_data']
        restructured_dict['binned_differences'] = data[gen][iteration]['binned_differences']
        series = pd.Series(restructured_dict, index=restructured_dict.keys())

        ga_results = pd.concat([ga_results, series], axis=1, ignore_index=True)
ga_results = ga_results.T.sort_values(by='dna_score', ascending=False, ignore_index=True)
ga_results

,generation,iteration,dna,dna_score,neuron_data,binned_differences
0,generation1,iteration0,"[102, 255, -174, -385, 226, 100, 71, -233, -33...",136,{'experimental': [Voltage is set to -60.0 and ...,"[[0, 0, 0, 0, 6, 6, 5, 6, 0, 0, 0, 0, 0, 0, 0,..."
1,generation0,iteration1,"[102, 255, -174, -385, 226, 100, 71, -233, -33...",136,{'experimental': [Voltage is set to -60.0 and ...,"[[0, 0, 0, 0, 6, 6, 5, 6, 0, 0, 0, 0, 0, 0, 0,..."
2,generation0,iteration8,"[295, 237, -360, -355, 215, 256, 312, -205, -8...",134,{'experimental': [Voltage is set to -60.0 and ...,"[[0, 0, 0, 0, 6, 6, 5, 6, 0, 0, 0, 0, 0, 0, 0,..."
3,generation1,iteration7,"[295, 233, -290, -364, 261, 256, 312, -205, -8...",134,{'experimental': [Voltage is set to -60.0 and ...,"[[0, 0, 0, 0, 6, 6, 5, 6, 0, 0, 0, 0, 0, 0, 0,..."
4,generation1,iteration6,"[393, 347, -169, -276, 152, 84, 71, -346, -324...",134,{'experimental': [Voltage is set to -60.0 and ...,"[[0, 0, 0, 0, 6, 6, 5, 6, 0, 0, 0, 0, 0, 0, 0,..."
5,generation1,iteration4,"[102, 347, -169, -385, 157, 78, 68, -346, -335...",134,{'experimental': [Voltage is set to -60.0 and ...,"[[0, 0, 0, 0, 6, 6, 5, 6, 0, 0, 0, 0, 0, 0, 0,..."
6,generation1,iteration1,"[393, 347, -191, -223, 157, 84, 68, -346, -324...",134,{'experimental': [Voltage is set to -60.0 and ...,"[[0, 0, 0, 0, 6, 6, 5, 6, 0, 0, 0, 0, 0, 0, 0,..."
7,generation1,iteration9,"[7, 247, -241, -388, 360, 381, 369, -81, -80, ...",134,{'experimental': [Voltage is set to -60.0 and ...,"[[0, 0, 0, 0, 6, 6, 5, 6, 0, 0, 0, 0, 0, 0, 0,..."
8,generation0,iteration3,"[7, 247, -241, -388, 360, 381, 369, -81, -80, ...",134,{'experimental': [Voltage is set to -60.0 and ...,"[[0, 0, 0, 0, 6, 6, 5, 6, 0, 0, 0, 0, 0, 0, 0,..."
9,generation0,iteration2,"[393, 347, -169, -276, 157, 84, 68, -346, -324...",134,{'experimental': [Voltage is set to -60.0 and ...,"[[0, 0, 0, 0, 6, 6, 5, 6, 0, 0, 0, 0, 0, 0, 0,..."


In [6]:
best_dna = ga_results.loc[0, 'dna']
print(best_dna)
diagnostic = {
        'show_dna_matrix' : False,
        'show_neuron_plots' : False,
        'show_difference_histogram' : False,
        'show_dna_scores': False
    }



if diagnostic['show_dna_matrix']:
                print("Currently loaded matrix ---")
                display_matrix(dna_matrix, NEURON_NAMES)

if diagnostic['show_dna_scores']:
                print(f'{dna_score=}: {curr_dna}')
            
if diagnostic['show_neuron_plots']:
                for condition in ['experimental', 'control']:
                    plot_neurons_interactive(neurons=neuron_data[condition], sq_wave=input_waves[0], go_wave=input_waves[1], show_u=False)
                    
if diagnostic['show_difference_histogram']:
                plot_binned_differences(binned_differences)

[102, 255, -174, -385, 226, 100, 71, -233, -335, -208, 343, 94, 177, -394, -227, -384, 246, 57, 323]
